In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [2]:
def scrape_1_page(i):
    url = "https://www.wildlifecrossing.net/california/observations/roadkill?page="+i
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')

    #get table headers
    col_labels = soup.find_all('tr')
    col_str = str(col_labels)
    all_header = []
    cleantext2 = BeautifulSoup(col_str, "lxml").get_text()
    all_header.append(cleantext2)

    df2 = pd.DataFrame(all_header)
    df3 = df2[0].str.split(',', expand=True)

    df3[0] = df3[0].str.strip('[\n')
    df3[5] = df3[5].str.strip(']')

    for i in range(1,6):
        df3[i] = df3[i].str.strip()


    #get table data
    rows = soup.find_all('tr')
    list_rows = []
    for row in rows:
        cells = row.find_all('td')
        str_cells = str(cells)
        clean = re.compile('<.*?>')
        clean2 = (re.sub(clean, '',str_cells))
        list_rows.append(clean2)

    # put table data into dataframe
    df = pd.DataFrame(list_rows)
    
    #split data into multiple columns
    df1 = df[0].str.split('\n', expand=True)

    ## strip meaningless symbols
    df1[6] = df1[6].str.strip(']')

    ##already done -->
    df1.drop(0, axis=1,inplace=True)


    for i in range(1,7):
        df1[i] = df1[i].str.strip(' ,')
    df1.reset_index(drop=True)

    #rearrange index after deleting column
    df1.columns = [np.arange(0,df1.shape[1])]

    # drop first row with ID =''
    df1.drop(df1[df1[0].isnull()==True].index,inplace=True) 


    #concactanate column labels and data 
    frames = [df3, df1]

    df4 = pd.concat(frames)

    # rename column with label
    df5 = df4.rename(columns=df4.iloc[0])
    df5.drop(0, axis=0,inplace=True)

    return df5



In [69]:
df = pd.DataFrame() 

#scrape page 1-71 (2010-2019)
for i in range (0,71):
    i = str(i)
    dftmp=scrape_1_page(i)
    frames = [df, dftmp]
    df_f = pd.concat(frames)
    df_f.index = [np.arange(0,df_f.shape[0])]
    df = df_f

In [70]:
print(df)

        ID       Observation Date         Category  \
0     77185      2019-04-21 18:00   Mammal (Large)   
1     77190      2019-04-03 07:15   Mammal (Large)   
2     77189      2019-04-02 04:15   Mammal (Large)   
3     77188      2019-04-01 12:00   Mammal (Large)   
4     77179      2019-04-01 11:00   Mammal (Small)   
5     77178      2019-04-01 11:00   Mammal (Small)   
6     77176      2019-03-31 13:30          Reptile   
7     77164      2019-03-30 12:30  Mammal (Medium)   
8     77162      2019-03-30 12:30             Bird   
9     77163      2019-03-30 12:30             Bird   
10    77182      2019-03-30 12:00   Mammal (Small)   
11    77181      2019-03-30 11:45   Mammal (Small)   
12    77180      2019-03-30 11:45   Mammal (Small)   
13    77167      2019-03-30 09:15   Mammal (Large)   
14    77161      2019-03-29 12:45        Amphibian   
15    77160      2019-03-29 12:30        Amphibian   
16    77166      2019-03-29 09:00   Mammal (Large)   
17    77158      2019-03-28 

In [73]:
df.to_csv('out.csv')

** as of Sep 2019, public webscraping is no longer available on California Roadkill Observation System